In [ ]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

In [ ]:
h2o.init()

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv').drop('id', axis=1)
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
test_id = test.pop('id')

In [ ]:
features = [col for col in test.columns if 'f' in col]

train['n_missing'] = train[features].isna().sum(axis=1)
test['n_missing'] = test[features].isna().sum(axis=1)

train['std'] = train[features].std(axis=1)
test['std'] = test[features].std(axis=1)

In [ ]:
df_train = h2o.H2OFrame(train)
df_test = h2o.H2OFrame(test)
df_train['claim'] = df_train['claim'].asfactor()

In [ ]:
HOURS = 4

aml = H2OAutoML(max_runtime_secs = int(3600*HOURS),
                exclude_algos = ['GLM', 'DeepLearning'],
                verbosity = 'info',
                seed =1)

aml.train(training_frame = df_train, y = 'claim')

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
submission = aml.predict(df_test).as_data_frame().set_index(test_id).reset_index().drop(columns = ['predict', 'p0']).rename(columns = {'p1':'claim'})
submission.to_csv('submission.csv', index = False)